# Lesson 02
## Парсинг HTML. BeautifulSoup, MongoDB

## 1 вариант
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru
и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). 
Получившийся список должен содержать в себе минимум:
* Наименование вакансии
* Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
* Ссылку на саму вакансию
* Сайт откуда собрана вакансия

In [31]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [32]:
def parsing_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(item_hh(item))
                
    return vacancy_date

In [33]:
def item_hh(item):

    vacancy_date = {}
    
 
    vacancy_name = item.find('a', {'class': 'bloko-link HH-LinkModifier'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_date['vacancy_name'] = vacancy_name
    
  
    salary = item.find('div',{'class':'vacancy-serp-item__compensation'})
    if not salary:
        salary_min=None
        salary_max=None
        salary_currency=None
    else:
        salary=salary.getText().replace(u'\xa0', u' ')
        salaries=salary.split('-')
        salary_min=salaries[0]
        if len(salaries)>1:
            salary_max=salaries[1]
        else:
            salary_max=''
        
        salary_currency = salary[2]
         
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    vacancy_link = item.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    
    vacancy_date['vacancy_link'] = vacancy_link 
    

    vacancy_date['site'] = 'hh.ru'
    
    return vacancy_date

In [34]:
def parsing_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'geo[c][0]': '15', \
        'geo[c][1]': '1', \
        'geo[c][2]': '9', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for item in vacancy_items:
                vacancy_date.append(item_superjob(item))
                
    return vacancy_date

In [35]:
def item_superjob(item):

    vacancy_date = {}
    
 
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    

   

    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_sarary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if is_check_sarary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = int(salary[0].getText() \
                                        .replace(u'\xa0', u''))
        elif is_check_sarary == 'от':
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = None
        else:
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = int(salary[2].getText() \
                                         .replace(u'\xa0', u''))     
            
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    

    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'
    

    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [36]:
def parsing_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parsing_hh(vacancy))
    vacancy_date.extend(parsing_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [37]:
vacancy = 'Python'
df = parsing_vacancy(vacancy)

In [38]:
df.loc[(df['salary_min'].notnull())&(df['salary_max'].notnull()),:]

,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site
916,Ведущий разработчик Python,150000.0,300000.0,₽,https://www.superjob.ru/vakansii/veduschij-raz...,www.superjob.ru
917,Разработчик Python,170000.0,200000.0,₽,https://www.superjob.ru/vakansii/razrabotchik-...,www.superjob.ru
925,Ведущий разработчик Python,150000.0,300000.0,₽,https://www.superjob.ru/vakansii/veduschij-raz...,www.superjob.ru
926,Разработчик Python,170000.0,200000.0,₽,https://www.superjob.ru/vakansii/razrabotchik-...,www.superjob.ru


In [39]:
df.loc[(df['site']== 'hh.ru'),:].head(3)

,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site
0,Программист Python,NaN,NaN,None,https://hh.ru/vacancy/36293188?query=Python,hh.ru
1,Fullstack Python Developer,NaN,NaN,None,https://hh.ru/vacancy/36222549?query=Python,hh.ru
2,Программист-разработчик python (midle),NaN,NaN,None,https://hh.ru/vacancy/35758988?query=Python,hh.ru


In [40]:
df.loc[(df['site']== 'hh.ru')&(df['salary_min'].notnull()),:].head(3)

,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site


In [ ]:
 #ПРОБЛЕМНАЯ ЧАСТЬ КОДА
    
    salary = item.find('div',{'class':'vacancy-serp-item__compensation'})
    if not salary:
        salary_min=None
        salary_max=None
        salary_currency=None
    else:
        salary=salary.getText().replace(u'\xa0', u' ')
        salaries=salary.split('-')
        salary_min=salaries[0]
        if len(salaries)>1:
            salary_max=salaries[1]
        else:
            salary_max=''
        
        salary_currency = salary[2]